In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import TimeSeriesSplit

In [2]:
df = pd.read_csv('J2D_train.csv')

In [3]:
df

,id,amount,status,created_at,user_id,money_back_date,transfer_type,send_at
0,4405,100.0,rejected,2020-06-20 16:46:28.409042+00,14799.0,NaN,regular,NaN
1,13107,100.0,approved,2020-08-19 16:20:22.768375+00,8020.0,2020-09-24 22:23:28.213168+00,regular,2020-08-26 16:20:22.631568+00
2,11984,50.0,approved,2020-08-11 13:23:12.220906+00,27793.0,2020-11-19 23:00:00+00,instant,2020-08-18 13:23:12.220516+00
3,11322,50.0,approved,2020-08-07 11:01:30.51934+00,15409.0,2020-08-29 22:12:23.812361+00,instant,2020-08-14 11:01:30.518974+00
4,16755,50.0,rejected,2020-09-18 13:08:34.789948+00,39701.0,NaN,regular,2020-09-25 13:08:34.765418+00
...,...,...,...,...,...,...,...,...
16100,13302,100.0,rejected,2020-08-20 07:23:33.760027+00,29247.0,NaN,instant,2020-08-27 07:23:33.735377+00
16101,13710,100.0,rejected,2020-08-24 00:33:58.853099+00,NaN,NaN,instant,2020-08-31 00:33:58.824223+00
16102,1522,100.0,approved,2020-05-02 08:54:49.586739+00,2562.0,2020-06-13 00:16:39.223405+00,regular,NaN
16103,7153,100.0,rejected,2020-07-08 12:56:16.163418+00,NaN,NaN,regular,NaN


In [4]:
df['ca_date'] = pd.to_datetime(df.created_at)

In [85]:
grup = pd.Grouper(key='ca_date', freq='M' )

dfagg = df.groupby(grup).agg({'amount':['sum','count','mean','median'],'user_id':['nunique']})

In [86]:
dfagg

amount                         user_id
                                sum count       mean median nunique
ca_date                                                            
2019-11-30 00:00:00+00:00       1.0     1   1.000000    1.0       1
2019-12-31 00:00:00+00:00   27297.0   289  94.453287  100.0     205
2020-01-31 00:00:00+00:00   21587.0   223  96.802691  100.0     159
2020-02-29 00:00:00+00:00   16653.0   184  90.505435  100.0     147
2020-03-31 00:00:00+00:00   23549.0   244  96.512295  100.0     183
2020-04-30 00:00:00+00:00   46093.0   473  97.448203  100.0     369
2020-05-31 00:00:00+00:00   79236.0   837  94.666667  100.0     636
2020-06-30 00:00:00+00:00  246026.0  2615  94.082600  100.0    1835
2020-07-31 00:00:00+00:00  328187.0  3601  91.137740  100.0    2418
2020-08-31 00:00:00+00:00  287633.0  3417  84.177056  100.0    2384
2020-09-30 00:00:00+00:00  336507.0  4221  79.722104  100.0    3323

In [152]:
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [88]:
dfagg.columns = ['am_total','am_count','am_mean','am_median','us_unique']

In [89]:
dfagg.columns

Index(['am_total', 'am_count', 'am_mean', 'am_median', 'us_unique'], dtype='object')

In [121]:
dfagg['pred'] = dfagg.am_total.shift(-1)
dfagg['diff'] = dfagg.am_total - dfagg.am_total.shift(1)

In [153]:
dfagg

,am_total,am_count,am_mean,am_median,us_unique,pred,diff
ca_date,,,,,,,
2019-11-30 00:00:00+00:00,1.0,1,1.000000,1.0,1,27297.0,NaN
2019-12-31 00:00:00+00:00,27297.0,289,94.453287,100.0,205,21587.0,27296.0
2020-01-31 00:00:00+00:00,21587.0,223,96.802691,100.0,159,16653.0,-5710.0
2020-02-29 00:00:00+00:00,16653.0,184,90.505435,100.0,147,23549.0,-4934.0
2020-03-31 00:00:00+00:00,23549.0,244,96.512295,100.0,183,46093.0,6896.0
2020-04-30 00:00:00+00:00,46093.0,473,97.448203,100.0,369,79236.0,22544.0
2020-05-31 00:00:00+00:00,79236.0,837,94.666667,100.0,636,246026.0,33143.0
2020-06-30 00:00:00+00:00,246026.0,2615,94.082600,100.0,1835,328187.0,166790.0
2020-07-31 00:00:00+00:00,328187.0,3601,91.137740,100.0,2418,287633.0,82161.0


In [148]:
# X = dfagg[6:-1]
X = dfagg[:-1].fillna(0)
y = X.pred

In [149]:
train_cols = ['am_total', 'am_count', 'am_mean', 'us_unique','diff']
train_cols = ['am_total', ]

In [150]:
regr = LGBMRegressor()
# regr = SVR(degree=3)
regr = LinearRegression()

regr.fit(X[train_cols],y)

LGBMRegressor()

In [151]:
regr.predict(dfagg[-1:][train_cols])

array([141276.8])

In [84]:
dfagg

,am_total,am_count,am_mean,am_median,us_unique,pred
ca_date,,,,,,
2019-11-24 00:00:00+00:00,1.0,1,1.000000,1.0,1,0.0
2019-12-01 00:00:00+00:00,0.0,0,NaN,NaN,0,0.0
2019-12-08 00:00:00+00:00,0.0,0,NaN,NaN,0,10126.0
2019-12-15 00:00:00+00:00,10126.0,106,95.528302,100.0,87,11091.0
2019-12-22 00:00:00+00:00,11091.0,119,93.201681,100.0,91,4540.0
2019-12-29 00:00:00+00:00,4540.0,48,94.583333,100.0,39,3640.0
2020-01-05 00:00:00+00:00,3640.0,37,98.378378,100.0,27,5870.0
2020-01-12 00:00:00+00:00,5870.0,60,97.833333,100.0,45,3665.0
2020-01-19 00:00:00+00:00,3665.0,38,96.447368,100.0,32,3020.0
